Importanto bibliotecas e arquivos e definindo listas:

In [83]:
import pandas as pd
import re

treinamento = pd.read_excel('Razer.xlsx')
teste = pd.read_excel('teste.xlsx')
teste2 = pd.read_excel('teste.xlsx')

#Cria índices numéricos pros dados
treinamento.index = range(len(treinamento))
teste.index = range(len(teste)) 
teste2.index = range(len(teste2)) 

pd.set_option('display.max_colwidth', -1) # Mostrar linha inteira na tabela

palavrasRemovidas = ['ao','de','da','do','que','qual','quai','por','para','pra','pro','com','em','eh','era','essa','esta','esse','este','isso','isto','razer']
caracInuteis = ['!','@','#','$','%','¨','&','*','(',')','-','_','+','=','§',',','.','<','>',':',';','/','?','°','|','\\','{','}','ª','[',']','º','×','–','—','•','…','‹','›','€']
subst=[('á|à|â|ã','a'),('é|è|ê','e'),('í|ì|î','i'),('ó|ò|ô|õ','o'),('ú|ù|û','u')]

p1 = 'https?:.[^ ]*' #tira links (até um espaço)
p2 = '@[^ ]*' #tira @pessoa (até um espaço)
p3 = '\n' #tira quebra de linha
p4 = '[0-9]' #tira os números
padraofull = "|".join([p1,p2,p3,p4]) #Combina os elementos da lista_pattern

Funções para limpar tabela:

In [84]:
#Função para limpar uma linha:
def limparLinha(linha):  

    #Transforma linha em lista de palavras:
    palavras_linha = re.findall(r'[^ ]+', linha)
    listaLimpa=[]

    #Primeiro tira acentos, padrões regex e plural (s no final). Depois a palavra é separada em novas palavras por carac.'s
    #que não forem letras (a-z ou ç) e dividida em palavras e simbolos.
    #As novas palavras que não estiverem em palavrasRemovidas e tiverem pelo menos 2 carac são retornadas
    #Os simbolos que não estiverem na lista caracInuteis são retornados

    for palavra in palavras_linha: 
        #Tira Acentos
        for comAcento,semAcento in subst:
            palavra = re.sub(comAcento, semAcento, palavra) 

        #Tirar padrões Regex
        palavra = re.sub(padraofull, '', palavra)      

        #Tira plural (apenas o s do final)
        palavra = re.sub('s$', '', palavra) #tirar plural    

        #Separar Simbolos e Emojis das palavras
        palSimbolos = re.findall('[^a-zç]',palavra)
        palLetras = re.findall('[a-zç]+',palavra)

        for i in palSimbolos:
            #Tira caractéres 
            if i not in caracInuteis:
                listaLimpa.append(i) 

        for j in palLetras:
            #Tira palavras da lista palavrasRemovidas
            if j in palavrasRemovidas:
                j = ''

            #Tira palavras com menos de 2 caracteres        
            if len(j) > 1:
                listaLimpa.append(j) 

    frase_Limpa = " ".join(listaLimpa)
    return frase_Limpa

#Função para pegar tabela e passar todas linhas pela função que limpa linha
def preProcessar(table):
    for n_linha in range(len(table)):
        linha = table.loc[n_linha,'Treinamento']   
        table.loc[n_linha,'Treinamento'] = limparLinha(linha)
    return table

Função para gerar tabela de frenquências de cada palavra da tabela de treinamento:

In [85]:
#Só o treinamento passa por essa função
def gerarTabelaFrequencias(table):
    freq={} #tabela com a palavra, Repet. Relev, Repet. Irrelev, Prob palavra ser Rel, Prob palavra ser Irrelev
    totalR,totalI = 0,0 #Total de palavras Rel, Total de palavras Irrel
    listaIndices = table.index.values
    
    #Cria lista freq de repetições Relev e Irrelev de cada palavra
    for n_linha in listaIndices:
        relevancia = table.loc[n_linha,'relevancia']
        linha = table.loc[n_linha,'Treinamento'] 
        palavras_linha = re.findall(r'[^ ]+', linha) #cria lista de palavras da linha
        
        for palavra in palavras_linha:
            if palavra not in freq:
                freq[palavra] = {'R':0,'I':0,'pR':0,'pI':0}
            if relevancia == 0:
                freq[palavra]['I'] += 1
            if relevancia == 1:
                freq[palavra]['R'] += 1
    
    #Conta a qtd. de palavras Relev. e Irrelev. (contando as repetições)
    for key,valor in freq.items():  
        totalR += valor['R']
        totalI += valor['I']
   
    #Calcula a prob da palavra ser relev e de ser irrelev
    for key,valor in freq.items():
        freq[key]['pR'] = (valor['R']+1)/(2*totalR+totalI)                
        freq[key]['pI'] = (valor['I']+1)/(2*totalI+totalR) 
        
    return freq,totalR,totalI

Função para calcular a probabilidade de cada tweet da tabela de teste ser relevante:

In [86]:
#Só o teste passa por essa função
def probLinha(table):
    listaIndices = table.index.values
    for n_linha in listaIndices:
        linha = table.loc[n_linha,'Treinamento']
        lista_linha = re.findall(r'[^ ]+', linha) # cria lista com as palavras da linha
        
        #Se a linha tiver links é bem mais provável que ela seja irrelevante:
        linha_original = teste2.loc[n_linha,'Treinamento']
        n_links = re.findall('https?:.[^ ]*', linha_original)
    
        pLR,pLI = 1,1 # Prob linha ser relevante e prob linha ser irrelevante
        for palavra in lista_linha:
            if palavra in freq:
                pLR *= (freq[palavra]['R']+1)/(2*totalR+totalI)                
                pLI *= (freq[palavra]['I']+1)/(2*totalI+totalR) 
            else:
                pLR *= 1/(2*totalR+totalI)                
                pLI *= 1/(2*totalI+totalR) 
    
        table.loc[n_linha,'pLR'] = pLR
        table.loc[n_linha,'pLI'] = pLI       
        if len(n_links) > 0:
            table.loc[n_linha,'pLI'] *= 1000
    return table

# Calculando probabilidade dos Tweets serem relevantes:

In [87]:
#Gerar tabela de probabilidade da tabela de treinamento
treinamentoLimpo = preProcessar(treinamento)
freq,totalR,totalI=gerarTabelaFrequencias(treinamentoLimpo)

#Analisar probabilidade de cada tweet da tabela de teste
testeLimpo = preProcessar(teste)
probs = probLinha(testeLimpo)

### Verificando a performance:

In [88]:
VerRel=probs[(probs["pLR"]>probs["pLI"])&(probs.relevancia==1)] #Relevantes realmente relevantes
VerIre=probs[(probs["pLR"]<probs["pLI"])&(probs.relevancia==0)] #Irrelevantes realmente irrelevantes
FalIre=probs[(probs["pLR"]<=probs["pLI"])&(probs.relevancia==1)] #Relevantes que foram dados como irrelevantes
FalRel=probs[(probs["pLR"]>=probs["pLI"])&(probs.relevancia==0)] #Irrelevantes que foram dados como relevantes

print('Verdadeiros Relevantes : ',len(VerRel))
print('Verdadeiros Irrelevantes : ',len(VerIre))
print('Relevantes dados como irrelevantes: ',len(FalIre))
print('Irrelevantes dados como relevantes : ',len(FalRel))
print()
print('Precisão de acerto dos relevantes:',100*round(len(VerRel)/len(VerRel+FalIre),4),'%')
print('Precisão de acerto dos irrelevantes:',100*round(len(VerIre)/len(VerIre+FalRel),4),'%')

Verdadeiros Relevantes :  11
Verdadeiros Irrelevantes :  179
Relevantes dados como irrelevantes:  1
Irrelevantes dados como relevantes :  9

Precisão de acerto dos relevantes: 91.67 %
Precisão de acerto dos irrelevantes: 95.21 %


#### Tweets que o código considera relevante:

In [89]:
indices = probs[(probs["pLR"]>probs["pLI"])].index.values
teste2.iloc[indices,:]

Treinamento  \
2    @bloodlustmary compra logitech, eu tô usando o meu da razer até comprar outro da logitech skmdmdmd\n\nesse meu da razer nunca deu problema permanente, mas já experimentei uma dose de não conectar o usb, mesmo com cabos e entradas boas hsidjsidjidjdi                                   
4    @chipart_info precisa nem perguntar,já tô enchendo o saco da amd,asus,nvidia,razer, toshiba, hiperx mastercooler e tudo mais                                                                                                                                                                
5    @deimomimis eu queria pegar qualquer bosta da razer oaaopekoaeka\nmas na questão do custo benefício to quase ficando com o motorola msm.                                                                                                                                                    
6    @endmosttt prefiro o razer chroma aushaush                                                                                                                                                                                                                                                  
7    @fuistalkeado mlk razer se quebra sozinho eu nem tenho culpa é o destino                                                                                                                                                                                                                    
10   @nicko_gg faaaaala galerinha. hoje eu vou trazer um gameplay da nova privada da razer!                                                                                                                                                                                                      
12   @razerbrazil me patrocina razer eu não tenho seguidores mas eu amo vcs a mó cota                                                                                                                                                                                                            
14   @tasogareriiku razer≠custo/benefício                                                                                                                                                                                                                                                        
15   @umdescrente amigo vi você falando que tá interessado em comprar um razer hammerhead,tenho um novo que ganhei em um campeonato de fifa,original,e quero vender  pq não vou usar,vendo por preço abaixo do mercado                                                                           
16   @vamojogarag @razerbrazil razer me patrocina pfv                                                                                                                                                                                                                                            
17   @wiuurz @phillybeatzu @razer @asus_rog @echogear @divoom @tweethue @nanoleaf tá bom desculpa, achei q tu ia ficar tão feliz quando eu te marcasse                                                                                                                                           
24   comprei o mouse da razer e o danado é bonito pra caralho                                                                                                                                                                                                                                    
44   hiago me da um mouse da razer de presente?                                                                                                                                                                                                                                                  
45   hoje bateu a vontade daquilo que começa com b... o belo desconto no razer meka                                                                                                                                    

#### Tweets relevantes que o código ignorou:

In [90]:
#Se deve ao fato da palavra 'pad' ter uma prob. extremamente baixa devido as várias repetições da palavra por spams
#Para corrigir podemos punir as linhas repetidas (ou remove-las) para não prejudicar palavras relevantes
FalIre

Treinamento  \
8  aquele mouse pad firefly ou polari corsair eu uso ele fica fino fino   

   relevancia           pLR           pLI  
8  1           2.492415e-43  2.534962e-42